<a href="https://colab.research.google.com/github/rezwanh001/Kaggle-Competition-Gendered-Pronoun-Resolution/blob/master/Gendered_Pronoun_Resolution_Version_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [0]:
# import os
# import pprint
# import tensorflow as tf

# if 'COLAB_TPU_ADDR' not in os.environ:
#   print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
# else:
#   tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#   print ('TPU address is', tpu_address)

#   with tf.Session(tpu_address) as session:
#     devices = session.list_devices()
    
#   print('TPU devices:')
#   pprint.pprint(devices)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# to access kaggle datasets
!pip install kaggle

# Math operations
!pip install numpy==1.15.0

#
!pip install https://github.com/fchollet/keras/archive/cudnn.zip

  HTTP error 404 while getting https://github.com/fchollet/keras/archive/cudnn.zip
  Could not install requirement https://github.com/fchollet/keras/archive/cudnn.zip because of error 404 Client Error: Not Found for url: https://codeload.github.com/keras-team/keras/zip/cudnn
Could not install requirement https://github.com/fchollet/keras/archive/cudnn.zip because of HTTP error 404 Client Error: Not Found for url: https://codeload.github.com/keras-team/keras/zip/cudnn for URL https://github.com/fchollet/keras/archive/cudnn.zip


In [5]:
# Colab's file access feature
from google.colab import files

# retrieve upload file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.jason into the folder where the API expects to to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/kaggle/kaggle.json 

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes
chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory


In [6]:
# list competitions
!kaggle competitions list


ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2490           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge       9877            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4083           False  
imagenet-object-localization-challenge         2029-12-31 07:00:00  Research         Knowledge         35           False  
competitive-data-science-predict-future-sales  2019-12-31 23:59:00  Playground           Kudos       2385           False  
two-sigma-financial-news                       2019-07-15 23:59:00  Featured          $100,000       2927           False  
LANL-Ear

In [7]:
#download gendered-pronoun-resolution data
!kaggle competitions download -c gendered-pronoun-resolution

  0% 0.00/79.0k [00:00<?, ?B/s]
100% 79.0k/79.0k [00:00<00:00, 69.5MB/s]
  0% 0.00/425k [00:00<?, ?B/s]
100% 425k/425k [00:00<00:00, 60.8MB/s]


In [7]:
###=============== Import necessary Libraries ==================
import os
import csv
import json
import string
import keras
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from math import floor
import spacy

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

import time
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate, Add, Flatten, CuDNNLSTM
from keras.models import Model
from keras import backend as K
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.engine.topology import Layer
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

from wordcloud import WordCloud, STOPWORDS
from collections import defaultdict


# keras libraries
from keras.models import Model, load_model,Sequential
from keras.layers import Dense, Input, Dropout,Bidirectional, GRU, Activation, concatenate, Embedding, SpatialDropout1D
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D ,GlobalMaxPool1D, GlobalAvgPool1D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
from keras import layers



Using TensorFlow backend.


In [8]:
##======== Load text and tokenize ====================
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'A few days later, Abigail complained that Elizabeth was pinching her and tearing at her bowels')

for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_)

A DET DT det
few ADJ JJ amod
days NOUN NNS npadvmod
later ADV RB advmod
, PUNCT , punct
Abigail PROPN NNP nsubj
complained VERB VBD ROOT
that ADP IN mark
Elizabeth PROPN NNP nsubj
was VERB VBD aux
pinching VERB VBG ccomp
her PRON PRP dobj
and CCONJ CC cc
tearing VERB VBG conj
at ADP IN prep
her ADJ PRP$ poss
bowels NOUN NNS pobj


In [0]:
####======================= Define necessary libraries ========================
def word_locate(sentence, location): 
    count_words = 0
    count_chars = 2 #2 is to count for the two spaces in the beginning
    for word in sentence.split():
        count_words += 1
        if location == count_chars:
            return word, count_words
        count_chars += len(word)
        count_chars += 1 #for space
        
def curr_prev_sentence(sentence, loc):
    current_sentence = ""
    prev_sentence = ""
    detect = 0
    count = 0
    for char in sentence:
        count += 1
        current_sentence += char
        if char == "." and detect == 0:
            prev_sentence = current_sentence 
            current_sentence = ""
        if char == "." and detect == 1:
            return current_sentence, prev_sentence
        if count == loc:
            detect = 1

def find_subject(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == "nsubj":
            return token.text
    return "none"            

In [10]:
text1 = "After a few years of almost no work -- although he was a guest star on Lou Grant and Charlie's Angels in the late 1970s, he once summed up the 1970s as 'I cried and did a lot of gardening' -- he was hired in 1979 for his best-known role, self-made millionaire Palmer Cortlandt on ABC's long-running soap opera All My Children. Initially hired for only one year, he remained on contract through 2009. For much of his first decade on the show, Palmer was a ruthless villain, totally possessive of his daughter, Nina and violently threatening his ex-wife Daisy with being attacked by dobermans when she came back from the dead."

text2 = "When onlookers expressed doubt, claiming that the Proctor family was well regarded in the community, the girl promptly came out of her trance and told them it was all for 'sport'. On March 29, 1692, Abigail Williams and Mercy Lewis again said they were being tormented by Elizabeth's spectre. A few days later, Abigail complained that Elizabeth was pinching her and tearing at her bowels, and said she saw Elizabeth's spectre as well as John's."

current, prev = curr_prev_sentence(text2, 360)
print(current)

candidate = find_subject(current)
print(candidate)

word, loc = word_locate(text2, 360) 
print(word, " ", loc)


 A few days later, Abigail complained that Elizabeth was pinching her and tearing at her bowels, and said she saw Elizabeth's spectre as well as John's.
Abigail
her   60


In [11]:
!ls

drive		 sample_submission_stage_1.csv	test_stage_1.tsv
gap-coreference  submission_1.csv		test_stage_1.tsv.zip
sample_data	 submission_2.csv


In [13]:
!unzip test_stage_1.tsv.zip

Archive:  test_stage_1.tsv.zip
  inflating: test_stage_1.tsv        


In [14]:
from google.colab import files

!git clone https://github.com/google-research-datasets/gap-coreference.git

#!wget 'https://github.com/google-research-datasets/gap-coreference/blob/master/gap-development.tsv'

#!files.download('https://github.com/google-research-datasets/gap-coreference/blob/master/gap-development.tsv') # then browse, select the files. It's then uploaded

# uploaded is now a dict containing "filename" -> Content

Cloning into 'gap-coreference'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), done.


In [12]:
!ls


drive		 sample_submission_stage_1.csv	test_stage_1.tsv
gap-coreference  submission_1.csv		test_stage_1.tsv.zip
sample_data	 submission_2.csv


In [0]:
#### ======================== load test_stage_1
with open('test_stage_1.tsv') as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    testA_X = []
    testB_X = []
    test_auX = []
    test_ids = []
    train_auX = [] #we do a small hack and fill up the aux train data twice, like train hopping 
    for row in reader:
        text = row['Text']
        textA = text.lower()
        test_auX.append(text.lower())#test aux is half size of train aux
        train_auX.append(text.lower()) #since test loop is running, why not fill up aux train in meantime 
        new_textA = textA
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(textA)):
            if char_idx == int(row['A-offset']):
                new_textA = new_textA[:char_idx+check] + "person_loc " + new_textA[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textA = new_textA[:char_idx+check] + "pronom_loc " + new_textA[char_idx+check:]
                check = 11
        testA_X.append(new_textA)
        textB = text.lower()
        new_textB = textB
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(textB)):
            if char_idx == int(row['B-offset']):
                new_textB = new_textB[:char_idx+check] + "person_loc " + new_textB[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textB = new_textB[:char_idx+check] + "pronom_loc " + new_textB[char_idx+check:]
                check = 11
        testB_X.append(new_textB)
        test_ids.append(row['ID'])

In [14]:
print(len(test_ids))

2000


In [0]:
########=============================== Load gap-development.tsv ===================
with open('/content/gap-coreference/gap-development.tsv') as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
#     print(*reader)
    train_X = []
    train_y = []
    for row in reader:
#         print(row)
        text = row["Text"]
        text = text.lower()
        train_auX.append(text)
        new_textA = text
        labelA = 0
        labelB = 0
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(text)):
            if char_idx == int(row['A-offset']):
                new_textA = new_textA[:char_idx+check] + "person_loc " + new_textA[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textA = new_textA[:char_idx+check] + "pronom_loc " + new_textA[char_idx+check:]
                check = 11
        if row['A-coref'] == 'TRUE':
            labelA = 1
        train_X.append(new_textA)
        train_y.append(labelA)
        new_textB = text
        label = 0
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(text)):
            if char_idx == int(row['B-offset']):
                new_textB = new_textB[:char_idx+check] + "person_loc " + new_textB[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textB = new_textB[:char_idx+check] + "pronom_loc " + new_textB[char_idx+check:]
                check = 11
        if row['B-coref'] == 'TRUE':
            labelB = 1
        train_X.append(new_textB)
        train_y.append(labelB)

In [16]:
print(len(train_X))

4000


In [0]:
###============= Tokenize 
maxlen = 220
embed_size = 500
max_features = 7000


In [18]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer_list = list(train_X)
tokenizer.fit_on_texts(tokenizer_list)
train_X = tokenizer.texts_to_sequences(train_X)
testA_X = tokenizer.texts_to_sequences(testA_X)
testB_X = tokenizer.texts_to_sequences(testB_X)
test_auX = tokenizer.texts_to_sequences(test_auX)
train_auX = tokenizer.texts_to_sequences(train_auX)
train_X = pad_sequences(train_X, maxlen=maxlen)
testA_X = pad_sequences(testA_X, maxlen=maxlen)
testB_X = pad_sequences(testB_X, maxlen=maxlen)
test_auX = pad_sequences(test_auX, maxlen=maxlen)
train_auX = pad_sequences(train_auX, maxlen=maxlen)


word_index = tokenizer.word_index
max_features = len(word_index)

print(max_features)

22243


In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

      
      
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):


        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)


    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())


        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [0]:
train_y = np.asarray(train_y)


In [0]:
##==========================Define CuDNNGRU model
def get_model():
  inp1 = Input(shape=(maxlen,))
  inp2 = Input(shape=(maxlen,))

  model1_out = Embedding(max_features, embed_size)(inp1)
  model1_out = Bidirectional(CuDNNGRU(256, return_sequences=True))(model1_out)
  model1_out = AttentionWithContext()(model1_out)
  model1_out = Dropout(0.1)(model1_out)

  model2_out = Embedding(max_features, embed_size)(inp2)
  model2_out = Bidirectional(CuDNNGRU(256, return_sequences=True))(model2_out)
  model2_out = AttentionWithContext()(model2_out)
  model2_out = Dropout(0.1)(model2_out)

  merged_out = keras.layers.Concatenate(axis=1)([model1_out, model2_out])

  merged_out = Dense(32, activation="relu")(merged_out)
  merged_out = Dropout(0.1)(merged_out)
  
  merged_out = Dense(1, activation="sigmoid")(merged_out)
  model = Model(inputs=[inp1,inp2], outputs=merged_out)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model


In [0]:
##==========================Define LSTM model
def get_model_1():
  inp1 = Input(shape=(maxlen,))
  inp2 = Input(shape=(maxlen,))

  model1_out = Embedding(max_features, embed_size)(inp1)
  model1_out = Bidirectional(LSTM(256, return_sequences=True))(model1_out)
  model1_out = AttentionWithContext()(model1_out)
  model1_out = Dropout(0.1)(model1_out)

  model2_out = Embedding(max_features, embed_size)(inp2)
  model2_out = Bidirectional(LSTM(256, return_sequences=True))(model2_out)
  model2_out = AttentionWithContext()(model2_out)
  model2_out = Dropout(0.1)(model2_out)

  merged_out = keras.layers.Concatenate(axis=1)([model1_out, model2_out])

  merged_out = Dense(32, activation="relu")(merged_out)
  merged_out = Dropout(0.1)(merged_out)
  
  merged_out = Dense(1, activation="sigmoid")(merged_out)
  model = Model(inputs=[inp1,inp2], outputs=merged_out)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model


In [0]:
def get_model_gru():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = SpatialDropout1D(0.2)(x) # 0.2 -> 0.1
    x = Bidirectional(GRU(80, return_sequences=True, activation='relu', dropout=0.1, recurrent_dropout=0.0))(x) # 80 -> 85
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

    return model


In [22]:
model = get_model()
# model = get_model_gru()
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 220)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 220)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 220, 500)     11121500    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embeddin

In [23]:
model.fit([train_X, train_auX], train_y, batch_size=512, epochs=100, validation_data=([train_X,train_auX], train_y))


# model.fit(train_X, train_y, epochs=20, batch_size=32, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 4000 samples, validate on 4000 samples
Epoch 1/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.6897 - acc: 0.5485 - val_loss: 0.6880 - val_acc: 0.5503
Epoch 2/100
4000/4000 [==============================] - 14s 4ms/step - loss: 0.6883 - acc: 0.5503 - val_loss: 0.6861 - val_acc: 0.5503
Epoch 3/100
4000/4000 [==============================] - 14s 4ms/step - loss: 0.6852 - acc: 0.5503 - val_loss: 0.6770 - val_acc: 0.5503
Epoch 4/100
4000/4000 [==============================] - 14s 4ms/step - loss: 0.6681 - acc: 0.5660 - val_loss: 0.6287 - val_acc: 0.7150
Epoch 5/100
4000/4000 [==============================] - 14s 4ms/step - loss: 0.6419 - acc: 0.6390 - val_loss: 0.6126 - val_acc: 0.6967
Epoch 6/100
4000/4000 [==============================] - 14s 4ms/step - loss: 0.5827 - acc: 0.7065 - val_loss: 0.5134 - val_acc: 0.7298
Epoch 7/100
4000/4000 [==============================] - 14s 4ms/step - loss: 0.5378 - a

In [46]:
pred_valA_y = model.predict([testA_X,test_auX], batch_size=512, verbose=1)
pred_valB_y = model.predict([testB_X,test_auX], batch_size=512, verbose=1)

2000/2000 [==============================] - 2s 826us/step


In [47]:
print(len(pred_valB_y), " ", len(pred_valA_y))

2000   2000


In [0]:
out_df = pd.DataFrame({"ID":test_ids})
out_df['A'] = [max(float(val),0.33) for val in list(pred_valA_y)]
out_df['B'] = [max(float(val),0.33) for val in list(pred_valB_y)]
out_df['NEITHER'] = [max((1-float(val)),0.33) for val in list(pred_valA_y)]


In [49]:
out_df.head(10)

,ID,A,B,NEITHER
0,development-1,0.449853,0.449853,0.550147
1,development-2,0.449853,0.449853,0.550147
2,development-3,0.449853,0.449853,0.550147
3,development-4,0.449853,0.449853,0.550147
4,development-5,0.449853,0.449853,0.550147
5,development-6,0.449853,0.449853,0.550147
6,development-7,0.449853,0.449853,0.550147
7,development-8,0.449853,0.449853,0.550147
8,development-9,0.449853,0.449853,0.550147
9,development-10,0.449853,0.449853,0.550147


In [0]:
import pandas as pd
subm_1 = pd.read_csv('/content/submission_1.csv')
subm_2 = pd.read_csv('/content/submission_2.csv')
subm_3 = pd.read_csv('/content/submission_3.csv')
subm_4 = pd.read_csv('/content/submission_4.csv')


In [69]:
subm_1.head(10)

,ID,A,B,NEITHER
0,development-1,0.988302,0.999060,0.330000
1,development-2,0.330000,0.330000,0.999436
2,development-3,0.663525,0.999387,0.336475
3,development-4,0.330000,0.583480,0.999978
4,development-5,0.330000,0.330000,0.999999
5,development-6,0.999999,0.999806,0.330000
6,development-7,0.330000,0.330000,0.966465
7,development-8,0.330000,0.796346,0.999729
8,development-9,0.330000,0.997234,0.998847
9,development-10,0.989855,0.330000,0.330000


In [70]:
subm_2.head(10)

,ID,A,B,NEITHER
0,development-1,0.999787,0.999783,0.330000
1,development-2,0.330000,0.330000,0.999556
2,development-3,0.995672,0.995730,0.330000
3,development-4,0.641542,0.641542,0.358458
4,development-5,0.641542,0.641542,0.358458
5,development-6,0.330000,0.330000,0.998124
6,development-7,0.999808,0.999792,0.330000
7,development-8,0.330000,0.330000,0.970400
8,development-9,0.966441,0.997570,0.330000
9,development-10,0.641542,0.641542,0.358458


In [71]:
subm_3.head(10)

,ID,A,B,NEITHER
0,development-1,0.449853,0.449853,0.550147
1,development-2,0.449853,0.449853,0.550147
2,development-3,0.449853,0.449853,0.550147
3,development-4,0.449853,0.449853,0.550147
4,development-5,0.449853,0.449853,0.550147
5,development-6,0.449853,0.449853,0.550147
6,development-7,0.449853,0.449853,0.550147
7,development-8,0.449853,0.449853,0.550147
8,development-9,0.449853,0.449853,0.550147
9,development-10,0.449853,0.449853,0.550147


In [72]:
subm_4.head(10)

,ID,A,B,NEITHER
0,development-1,1.0,1.0,1.0
1,development-2,1.0,1.0,1.0
2,development-3,1.0,1.0,1.0
3,development-4,1.0,1.0,1.0
4,development-5,1.0,1.0,1.0
5,development-6,1.0,1.0,1.0
6,development-7,1.0,1.0,1.0
7,development-8,1.0,1.0,1.0
8,development-9,1.0,1.0,1.0
9,development-10,1.0,1.0,1.0


In [0]:
import pandas as pd
answers = pd.read_csv('/content/submission_1.csv', dtype={'A-coref': int, 'B-coref': int})
answers.rename(columns={'A-coref': 'A', 'B-coref': 'B'}, inplace=True)
answers['NEITHER'] = answers.eval('1 - A - B')
answers[['ID', 'A', 'B', 'NEITHER']].to_csv("answers.csv", index=False)

In [0]:
answers[['ID', 'A', 'B', 'NEITHER']].to_csv("/content/drive/My Drive/Kaggle Competition: Gendered Pronoun Resolution /answers.csv", index=False)

In [74]:
answers.head(10)

,ID,A,B,NEITHER
0,development-1,0.988302,0.999060,-0.987362
1,development-2,0.330000,0.330000,0.340000
2,development-3,0.663525,0.999387,-0.662912
3,development-4,0.330000,0.583480,0.086520
4,development-5,0.330000,0.330000,0.340000
5,development-6,0.999999,0.999806,-0.999805
6,development-7,0.330000,0.330000,0.340000
7,development-8,0.330000,0.796346,-0.126346
8,development-9,0.330000,0.997234,-0.327234
9,development-10,0.989855,0.330000,-0.319855


In [59]:
print(max(subm_1['A']), max(subm_2['A']), max(subm_3['A']))

1.0 0.9999995827674866 0.4498527944087982


In [61]:
print(max(subm_1['B']), max(subm_2['B']), max(subm_3['B']))

1.0 0.9999995231628418 0.4498527944087982


In [62]:
print(max(subm_1['NEITHER']), max(subm_2['NEITHER']), max(subm_3['NEITHER']))

0.9999999701976776 1.0 0.5501472055912018


In [0]:
out_df['A'] = max(max(subm_2['A']), max(subm_3['A']))
out_df['B'] = max(max(subm_2['B']), max(subm_3['B']))
out_df['NEITHER'] = max(max(subm_1['NEITHER']), max(subm_3['NEITHER']))

In [0]:
out_df.to_csv("submission_5.csv", index=False)

In [65]:
!ls '/content/drive/My Drive/Kaggle Competition: Gendered Pronoun Resolution '

 gendered-pronoun-resolution
'Gendered Pronoun Resolution: Version-04.ipynb'
 submission_1.csv
 submission_2.csv
 submission_3.csv
 submission_4.csv
'Top 3 NLP Libraries Tutorial( NLTK+spaCy+Gensim).ipynb'


In [0]:
out_df.to_csv("/content/drive/My Drive/Kaggle Competition: Gendered Pronoun Resolution /submission_5.csv", index=False)

In [75]:
!kaggle competitions submit -c gendered-pronoun-resolution -f answers.csv -m "V-06"

100% 116k/116k [00:03<00:00, 38.7kB/s]
Successfully submitted to Gendered Pronoun Resolution